# Assignment 4
* Julian Boaz
* CAP4611
* Due 11/21/2021
---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras_tuner import RandomSearch
#from keras_tuner.engine.hyperparameters import HyperParameters
import matplotlib.pyplot as plt

In [2]:
trainData = pd.read_csv('../input/assignment4new/train.csv')
testData = pd.read_csv('../input/assignment4new/eval.csv')

In [3]:
X = trainData.drop(['label','id'], axis=1)
y = trainData['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [4]:
#print(X_train.shape)
#print(X_test.shape)

In [5]:
#X_train = X_train.values.reshape(48000,28,28)
#X_test = X_test.values.reshape(12000,28,28)
#X_train.shape

In [6]:
#plt.gray()
#plt.imshow(X_train[1])


In [7]:
X_train = X_train.values.reshape(42000,28,28,1)
X_test = X_test.values.reshape(18000,28,28,1)

In [8]:
#X_train = X_train / 255
#X_test = X_test / 255
#y_train = y_train.to_numpy()
#y_train.shape
#y_train

In [9]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (28,28,1), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(256, (3,3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(10, activation = 'softmax'))

2021-11-22 01:34:50.004436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 01:34:50.114027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 01:34:50.115022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 01:34:50.117033: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [10]:
checkpoint = ModelCheckpoint('test_model5.h5', monitor='val_accuracy', save_best_only = True)

In [11]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) 
history = model.fit(X_train, y_train, epochs = 150, validation_data=(X_test, y_test), callbacks=[checkpoint], batch_size = 80) #increase batch size and see if it works

2021-11-22 01:34:53.138681: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


2021-11-22 01:34:54.936438: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


525/525 [==============================] - 12s 8ms/step - loss: 0.9860 - accuracy: 0.6741 - val_loss: 0.5224 - val_accuracy: 0.7972
Epoch 2/150
525/525 [==============================] - 3s 6ms/step - loss: 0.5747 - accuracy: 0.7877 - val_loss: 0.4435 - val_accuracy: 0.8408
Epoch 3/150
525/525 [==============================] - 3s 6ms/step - loss: 0.4804 - accuracy: 0.8261 - val_loss: 0.3662 - val_accuracy: 0.8588
Epoch 4/150
525/525 [==============================] - 4s 7ms/step - loss: 0.4277 - accuracy: 0.8446 - val_loss: 0.3540 - val_accuracy: 0.8698
Epoch 5/150
525/525 [==============================] - 3s 6ms/step - loss: 0.3933 - accuracy: 0.8582 - val_loss: 0.2991 - val_accuracy: 0.8897
Epoch 6/150
525/525 [==============================] - 3s 6ms/step - loss: 0.3707 - accuracy: 0.8674 - val_loss: 0.3177 - val_accuracy: 0.8867
Epoch 7/150
525/525 [==============================] - 4s 7ms/step - loss: 0.3443 - accuracy: 0.8774 - val_loss: 0.3089 - val_accuracy: 0.8868
Epoch 8/15

In [12]:
testDataShaped = testData.drop(['id'], axis=1).values.reshape(10000,28,28,1)

model.load_weights('./test_model5.h5')

print(model.evaluate(X_test, y_test))

modelPreds = model.predict(testDataShaped, verbose=0)
modelPreds.shape

563/563 [==============================] - 2s 4ms/step - loss: 0.2000 - accuracy: 0.9353
[0.1999722123146057, 0.9352777600288391]


(10000, 10)

In [13]:
labels = [np.argmax(x) for x in modelPreds]
output = pd.DataFrame({
    'id': testData['id'],
    'label': labels
})

In [14]:
output.to_csv('94.csv', index=False)
print(output.to_string())

        id  label
0        0      8
1        1      4
2        2      0
3        3      1
4        4      9
5        5      5
6        6      4
7        7      7
8        8      4
9        9      3
10      10      8
11      11      6
12      12      5
13      13      2
14      14      1
15      15      9
16      16      4
17      17      4
18      18      5
19      19      4
20      20      5
21      21      5
22      22      1
23      23      3
24      24      4
25      25      4
26      26      3
27      27      9
28      28      8
29      29      8
30      30      6
31      31      5
32      32      8
33      33      2
34      34      7
35      35      2
36      36      2
37      37      1
38      38      9
39      39      3
40      40      4
41      41      9
42      42      8
43      43      7
44      44      6
45      45      1
46      46      6
47      47      7
48      48      9
49      49      8
50      50      6
51      51      3
52      52      3
53      53      1
54      54